In [708]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
import sympy as sym
sym.init_printing(use_unicode=True)

In [709]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [710]:
def GetHermite(n,x):
    if n==0:
        poly = 1
    elif n==1:
        poly = 2*x
    else:
        poly = 2*x*GetHermite(n-1,x)-2*(n-1)*GetHermite(n-2,x)
   
    return sym.expand(poly,x)

In [711]:
def GetDHermite(n,x):
    Pn = GetHermite(n,x)
    return sym.diff(Pn,x,1)

In [712]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [713]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [714]:
def GetAllRootsGHer(n):

    xn = np.linspace(-1*(4*n+1)**(1/2),(4*n+1)**(1/2),100)
    
    Hermite = []
    DHermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermite(i,x))
        DHermite.append(GetDHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n],'numpy')
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

In [715]:
def GetWeightsGHer(n):

    Roots = GetAllRootsGHer(n)

    DHermite = []
    
    for i in range(n):
        DHermite.append(GetHermite(i,x))
    Dpoly = sym.lambdify([x],DHermite[n-1],'numpy')
    Weights = (2**(n-1)*np.math.factorial(n)*np.sqrt(np.pi))/(((n)**2)*(Dpoly(Roots))**2)
    
    return np.array(Weights)

In [716]:
GetAllRootsGHer(20)

array([-5.38748089, -4.60368245, -3.94476404, -3.34785457, -2.78880606,
       -2.254974  , -1.73853771, -1.23407622, -0.73747373, -0.24534071,
        0.24534071,  0.73747373,  1.23407622,  1.73853771,  2.254974  ,
        2.78880606,  3.34785457,  3.94476404,  4.60368245,  5.38748089])

In [717]:
GetWeightsGHer(20)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17192\3420257470.py:10: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  Weights = (2**(n-1)*np.math.factorial(n)*np.sqrt(np.pi))/(((n)**2)*(Dpoly(Roots))**2)


array([2.22939365e-13, 4.39934099e-10, 1.08606937e-07, 7.80255648e-06,
       2.28338636e-04, 3.24377334e-03, 2.48105209e-02, 1.09017206e-01,
       2.86675505e-01, 4.62243670e-01, 4.62243670e-01, 2.86675505e-01,
       1.09017206e-01, 2.48105209e-02, 3.24377334e-03, 2.28338636e-04,
       7.80255648e-06, 1.08606937e-07, 4.39934099e-10, 2.22939365e-13])

In [718]:
n = 10
I = 0
def function (x):
    return (1/np.pi)**(1/2)*(2*x**4)
roots = GetAllRootsGHer(n)
weights = GetWeightsGHer(n)

for i in range(n):
    
    I += weights[i]*function(roots[i])

print(I)

1.5000000000291789


C:\Users\ASUS\AppData\Local\Temp\ipykernel_17192\3420257470.py:10: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  Weights = (2**(n-1)*np.math.factorial(n)*np.sqrt(np.pi))/(((n)**2)*(Dpoly(Roots))**2)
